In [ ]:
import pandas as pd
import tqdm
import tsp

In [ ]:
tsp.run('NN', 'i_txt/kroE100.txt')

In [ ]:
df = pd.read_csv('instancias.csv')
df_instancias = df[df['dim'] < 200]
df_instancias.head()

**Experimento : Rendimiento de las heurísticas golosas**

En este experimento se busca evaluar el rendimiento de las heurísticas golosas implementadas. Se busca cuantificar qué tan buenas son estas heurísticas para encontrar una solución y qué tanto se aproxima esta la óptima. 

In [ ]:
df_greedy = pd.DataFrame()

GREEDY = ['NN', 'FI', 'AGM']

for _ in range(5):
    for g in tqdm.tqdm(GREEDY):
        for _, row in df_instancias.iterrows():
            
            instancia = row['instancia']
            file = f'./i_txt/{instancia}.txt'
            costo, tiempo = tsp.run(g, file)
            
            entry = {
                'instancia' : row['instancia'],
                'dim' : row['dim'],
                'heuristica' : g,
                'tiempo' : tiempo,
                'costo' : costo,
                'gap' : (costo - row['inf']) / row['inf'],
            }
            df_greedy = df_greedy.append(entry, ignore_index=True)
df_greedy.to_csv('greedy.csv', index=False)

**Experimento : TabúSearch - Optimización de parámetros**

- Sería interesante evaluar el porcentaje de caminos totales que estamos guardando. Para este experimento vamos a correrlo sobre `TS-C`.
- La cantidad de iteraciones debería ser por lo menos la cantidad de memoria?Mi razonamiento es que en cada iteracion de tabu search a lo sumo guardamos un circuito en la memoria, luego, para que 
- Hay que definir el vecindario 2opt.

In [ ]:
df_instancias_low = df_instancias[df_instancias['dim'] <= 100]
df_tabu = pd.DataFrame(columns=['instancia','dim','heuristica','tiempo', 'costo', 'gap', 'iteraciones', 'memoria', 'vecindad'])
df_tabu = df_tabu.astype({'dim': 'int64', 'tiempo': 'float64', 'costo': 'int64', 'gap':'float64', 'iteraciones': 'int64', 'memoria': 'int64', 'vecindad': 'int64'})
HEUR = ['TS-C', 'TS-S']

VECINDAD = [10, 20, 50, 80, 100]
MEMORIA = [50, 100, 200, 500, 1000]
ITERACIONES = [50, 100, 200, 500, 1000]

for _ in range(3):
    for h in tqdm.tqdm(HEUR):
        for _, row in df_instancias_low.iterrows():
            for vec in VECINDAD:
                for it in ITERACIONES:
                    for mem in MEMORIA:
                        if mem > it:
                            continue
                        
                        instancia = row['instancia']
                        file = f'./i_txt/{instancia}.txt'
                        costo, tiempo = tsp.run(h, file, it, mem, vec)
                        
                        entry = {
                            'instancia' : instancia,
                            'dim' : row['dim'],
                            'heuristica' : h,
                            'tiempo' : tiempo,
                            'costo' : costo,
                            'gap' : (costo - row['inf']) / row['inf'],
                            'iteraciones' : it,
                            'memoria' : mem,
                            'vecindad' : vec,
                        }
            
                        df_tabu = df_tabu.append(entry, ignore_index=True)
            df_tabu.to_csv('tabu_parcial.csv', index=False)
df_tabu.to_csv('tabu.csv', index=False)